In [1]:
%load_ext autoreload
%autoreload 15


In [ ]:
# !pip3 install azure-cosmos

In [ ]:
# !pip3 install comtypes

In [ ]:
# !pip3 install pdfminer.six

In [2]:
import json
import re
from bs4 import BeautifulSoup
import os
import requests
from pprint import pprint
import uuid
from statistics import mode
from nltk.corpus import stopwords
from pdf2txt import convert_pdf
# !pip install tika
import tika
from definitions_v3 import *

/opt/app/RB/.local/lib/python3.6/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.2) or chardet (4.0.0) doesn't match a supported version!
  RequestsDependencyWarning)


In [3]:
pointsRE_heading=re.compile('(?:\s*\([a-z]{1,3}\)|[A-Z]{1}\s+[a-zA-Z0-9_\s]{5})')
stop_words = list(set(stopwords.words('english')))
y_cordinate_Re=re.compile('top:(\d{1,5})px')
height_re=re.compile('height:(\d{1,5})px')

In [4]:
def check_underline(positions,y_cord_text):
    positions=[i for i in positions if y_cord_text<=i< y_cord_text+16]
    if positions:
        return True
    else:
        return False
    

In [5]:
def get_pages_no(soup):
    total_items = int(soup.find('span', id='NewReleases_total').text)
    items_per_page = int(soup.find('span', id='NewReleases_end').text)
    return round(total_items/items_per_page)

In [6]:
from pdfminer.high_level import extract_pages

# print(len(list(extract_pages('test/Chubb16-250-1019 Chubb EBM Business Pack Product Disclosure Statement (PDS) and Policy Wording.pdf'))))

In [7]:
def get_underlines(soup):
    positions=[]
    for span in soup.find_all('span'):
        y_cord_list=y_cordinate_Re.findall(str(span))
        if y_cord_list:
            y_cord=int(y_cord_list[0])
        else:
            continue
        style="position:absolute; border: black 1px solid" in str(span)
        height_px_li=height_re.findall(str(span))
        if height_px_li:
            height_px=height_px_li[0]
        else:
            continue
        height=int(height_px)<15
        if all([style,height]):
            positions.append(y_cord)
    return list(set(positions))

In [8]:
def ext_extract(text_list,language):
    ext_flag=False
    if language=='english':
        ext_lis=['extension']
    else:
        ext_lis=['extensión','extensiones']
   
    ext_count=0
    position_list=[]
    position=0
    for text in text_list:
        # if any two of 'exclusion ,condition or extension is present in the text, that text is omitted from the logic'
        bool_list=['condition' in text.lower(),'exclusion' in text.lower(),'extension' in text.lower()]
        if sum(bool_list)>=2:
            continue
        for i in ext_lis:
            for match in re.finditer(i, text.lower()):
                position_list.append(position+match.start())               
                ext_count+=1
        position+=len(text)
    if ext_count>=1:
        ext_flag=True
    return(ext_flag,ext_count,position_list)

In [9]:
def excl_extract(text_list,language):
    exclusion_flag=False
    if language=='english':
        excl_lis=['exclud','not cover','except','does not mean','not includ','exclusion']
    else:
        excl_lis=['excepto','excepción','no in cluido','exclusión','excluidos','excluirlo','excluyendo','exclusiones','exclusion','excluyen','excluyentes']
   
    exclusion_count=0
    position_list=[]
    position=0
    for text in text_list:
        # if any two of 'exclusion ,condition or extension is present in the text, that text is omitted from the logic'
        bool_list=['condition' in text.lower(),'exclusion' in text.lower(),'extension' in text.lower()]
        if sum(bool_list)>=2:
            continue
        for i in excl_lis:
            for match in re.finditer(i, text.lower()):
                position_list.append(position+match.start())               
                exclusion_count+=1
        position+=len(text)
    if exclusion_count>=1:
        exclusion_flag=True
    return(exclusion_flag,exclusion_count,position_list)

In [10]:
def cond_extract(text_list,language):
    cond_flag=False
    if language=='english':
        cond_lis=['condition']
    else:
        cond_lis=['condición','condiciones','condicionado']
   
    cond_count=0
    position_list=[]
    position=0
    for text in text_list:
        # if any two of 'exclusion ,condition or extension is present in the text, that text is omitted from the logic'
        bool_list=['condition' in text.lower(),'exclusion' in text.lower(),'extension' in text.lower()]
        if sum(bool_list)>=2:
            continue
        for i in cond_lis:
            for match in re.finditer(i, text.lower()):
                position_list.append(position+match.start())               
                cond_count+=1
        position+=len(text)
    if cond_count>=1:
        cond_flag=True
    return(cond_flag,cond_count,position_list)

In [11]:
def font_extraction(soup):
    fontsizes=[]
    for divs in soup.findAll('div'):
        for j in divs.find_all('span'):
            ext_size=re.findall(r'font-size:(.*)px">',str(j))
            if ext_size:
                fontsizes.append(int(ext_size[0]))
    return(fontsizes)

In [12]:
def stopword_check(word,text,language):
    if language=='spanish':
        stop_words = list(set(stopwords.words('spanish')))
    else:
        stop_words = list(set(stopwords.words('english')))
        transition="although  instead  whereas  despite  conversely  otherwise  however moreover  likewise  comparatively  correspondingly  similarly  furthermore  additionallyver  rather  nevertheless  nonetheless  regardless  notwithstanding consequently  therefore  thereupon  forthwith  accordingly  henceforth"
        transition_words=transition.split()
        transition_words
        stop_words.extend(transition_words)

    if exclusion_check(text,language):
        return True
    if word.lower() in stop_words:
        return False
    else:
        return True

In [13]:
def exclusion_check(text,language):
    if language=='spanish':
        exclusion_bag=['excepto','excepción','no in cluido','exclusión','excluidos','excluirlo','excluyendo',' excluyen','exclusiones','exclusiones','exclusion','excluyentes']
    else:
        exclusion_bag=['exclusion','excluded','not covered','will not cover','will not pay']
    for word in exclusion_bag:
        if word in text.lower():
            return True
    return False
    

In [14]:
def criteria_check(text,language):
    if language=='spanish':
        exclusion_bag=['condición','condiciones','condicionado']
    else:
        exclusion_bag=['condition','conditions']
    for word in exclusion_bag:
        if word in text.lower():
            return True
    return False
    

In [15]:
def ext_check(text,language):
    if language=='english':
        ext_bag=['extension']
    else:
        ext_bag=['extensión','extensiones']
    for word in ext_bag:
        if word in text.lower():
            return True
    return False
    

In [16]:
start=re.compile('.*\d{1,3}.*(?:\-|\.|–)')
def clean(text):
    text=text.replace('\n','').replace('\t',' ')
    text=start.sub('',text).strip()
    

    return(text)
def clean_pharses(phrases):
    phrases=[clean(i) for i in phrases if len(i)> 3]
    phrases=list(set(phrases))
    return phrases

In [17]:
def create_local_index(path,subdir,language,country):
    pointsRE_heading=re.compile('(?:\s*\([a-z]{1,3}\)|(?<![A-z])[A-Z]{1}\s+[a-zA-Z0-9_\s]{5})')
    pageNumRE=re.compile('Page\s*(\d{1,3})',re.IGNORECASE)
    neglect_def=['policy','insured','schedule']
    local_indexed={}
    previous_span=''
    bold_phrases_indexed={}
    for file in os.listdir(path):
        if file.endswith('.pdf'):
            print(file)
            global file_count
            file_count+=1
            
            print("COUNT",file_count)
            try:
                pg_no=len(list(extract_pages(os.path.join(path,file))))
            except:
                pg_no=0
            try:
                html=convert_pdf(os.path.join(path,file),'html')
            except:
                continue
            soup = BeautifulSoup(html, 'html5lib')
            underline_positions=get_underlines(soup)
            try:
                fontsizes=font_extraction(soup)
                file_font_size_mode=mode(fontsizes)
            except:
                file_font_size_mode=8
                
            text_para,text_plain_para='',''
            text_lis,text_plain_lis,bold_lis,page_list,=[],[],[],[]
            bold=False
            sub_page_def_list=[]
            try:
                definitions=def_extraction(os.path.join(path,file))
            except:
                continue
            definitions={key:value for key,value in definitions.items() if key.lower().strip() not in neglect_def}
            def_terms=list(definitions.keys())
#             print(len(def_terms),'LEN DEFINITIONS')
#             print(definitions)
            definition_text=''
            header_match_object=(0,'',False)
            cond_header_match_object=(0,'',False)
            ext_header_match_object=(0,'',False)
            second_category=False
            head_found=False
            def_flag=False
            cond_head_found=False
            ext_head_found=False
            single_page_head_found=False
            condition_text=''
            ext_text=''
            excl_text=''
            previous_pg_num=[]
            def_in_page=[]
            cond_single_page_head_found=False
            ext_single_page_head_found=False
            
            font_size=file_font_size_mode

            for divs in soup.findAll('div'):
                div_text_list=[span.text for span in divs.find_all('span') ]
                page_str=str(divs.find_all('a'))
                page_num_results=pageNumRE.findall(page_str)
                if page_num_results:
                    pagenum=page_num_results[0]
#                     print(pagenum)
                    if pagenum!=previous_pg_num:
                        sub_definitions_in_page=[]
                    previous_pg_num=pagenum
#                     definition_text=''
                    
                    #only for pdfs
                    if(int(pagenum) in [1,2]) and 'Wording' in file and pg_no >20:
#                     print('Continued on wording 2')
                        continue

                    if text_lis:
                        text_lis=text_lis[:-1]
                        text_para=''.join(text_lis)
                        text_plain_lis=text_plain_lis[:-1]
                        text_plain_para=''.join(text_plain_lis)
                        if(len(re.findall('\.',text_para))>(len(text_para)/3) or (len(re.findall('\d',text_para))>80 and int(pagenum)<4 and  pg_no>20 and 'content' in text_para.lower())) :
                            print('PASS')
                            print(text_plain_para)
                            text_para,text_plain_para=' ',' '
                            text_lis,text_plain_lis=[],[]
                            single_page_head_found=False
                            head_found=False
                            cond_head_found=False
                            ext_head_found=False
                            continue
                        
                        else:
                            bold_lis=clean_pharses(bold_lis)
                            pagenum=int(pagenum)-1
                            excl_flag,excl_count,excl_pos_lis=excl_extract(text_plain_lis,language)
                            cond_flag,cond_count,cond_pos_lis=cond_extract(text_plain_lis,language)
                            ext_flag,ext_count,ext_pos_lis=ext_extract(text_plain_lis,language)
                            if int(pagenum)==1:
                                if head_found or single_page_head_found or excl_count>0:
#                                     print(pagenum,True,'!!!')
                                    excl_flag=True
                                else:
                                    excl_flag=False
                            else:
                                if head_found or single_page_head_found:
#                                     print(pagenum,True,'!!!',excl_count)
                                    excl_flag=True
#                                     print(excl_flag,'EXCL')
#                                 else:
#                                     print(pagenum,excl_flag,'\n')
# #                             print(excl_text)
#                             print(condition_text)
                            if cond_head_found or cond_single_page_head_found or cond_count>0:
                                cond_flag=True
#                                 print('COND','TRUE')
                            else:
#                                 print('COND','FALSE')
                                cond_flag=False
    
                            if ext_head_found or ext_single_page_head_found or ext_count>0:
                                ext_flag=True
#                                 print('EXT','TRUE',pagenum)
                            else:
#                                 print('EXT','FALSE',pagenum)
                                ext_flag=False
#                             def_in_page=[{'name':k,'text':v[0] for k,v in definitions.items() if v[1]=pagenum}]
                            for term,defs in definitions.items():
#                                 print(defs[1])
                                if defs[1]==pagenum:  ###defs[1] is the pagenumber
                                    definition_text=definition_text+'          '+term+' '+defs[0]
                                    def_in_page.append({'name':term,'text':defs[0]})
#                                     def_in_page{'name':}
#                             print(definition_text)
                            if definition_text:
                                def_flag=True
                            
                            if def_flag:
                                def_search_flag=True
#                                 print('DEF','TRUE')
                            else:
#                                 print('DEF', 'FALSE')
                                def_search_flag=False   
                            
                            if pg_no>2:
                                endorsements=False
                            else:
                                endorsements=True
#                             print(endorsements,'ENDORSEMENTS!!!')
                            if text_para in local_indexed.keys():
                                text_para=text_para+' '
                            
                            local_indexed[text_para]=(file,pagenum,bold_lis,text_plain_para,excl_flag,excl_count,excl_pos_lis,subdir,sub_page_def_list,country,language,definition_text,def_search_flag,endorsements,excl_text,condition_text,cond_flag,cond_count,cond_pos_lis,def_in_page,ext_text,ext_flag,ext_count,ext_pos_lis)
#                             print(sub_page_def_list,'DEF LIST!!!!!1')
                            sub_page_def_list=[]
                            page_list.append(pagenum)
                            text_para,text_plain_para='',''
                            text_lis,text_plain_lis,bold_lis=[],[],[]
                            definition_text=''
                            cond_single_page_head_found=False
                            ext_single_page_head_found=False
                            condition_text=''
                            ext_text=''
                            excl_text=''
                            single_page_head_found=False
                            def_in_page=[]
                            def_flag=False
                for span in divs.find_all('span'):

                    bold=False
                    upper=False
                    bullet=False
                    def_flag=False
                    span_position=div_text_list.index(span.text)
                    
                    if "Bold" in str(span) or 'CIDFont+F3' in str(span):
                        bold_lis.append(span.text)
                        bold=True
                    if span.text.isupper():
                        upper=True
                    font_family_match=re.findall(r"font-family: b'(.*)';",str(span))
                    if font_family_match:
                        font_family=font_family_match[0]
                    else:
                        font_family=''
#                     print(font_family)
                    font_size_match=re.findall(r'font-size:(.*)px">',str(span))
                    if font_size_match:
                        font_size=int(font_size_match[0])
                    if pointsRE_heading.findall(span.text):
                        bullet=True
#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
                    y_cord_list=y_cordinate_Re.findall(str(divs))
                    if y_cord_list:
                        y_cord=int(y_cord_list[0])
                    underlined_text=check_underline(underline_positions,y_cord)
                    if span.text.split('\n')[0]!='' and span.text.split('\n')[0]!=' ':
                        head_check_text=span.text.split('\n')[0]
                    elif len(span.text.split('\n'))>1:
                        head_check_text=span.text.split('\n')[1]
                    else:
                        head_check_text=''
##############################3  
                    if str(previous_span).endswith('<br/></span>'):
                        span_position=0
                    ##################INSERT FUNCTION####################z
                    #EXCLUSION 
                    if  span_position==0 and (bold or font_size>= file_font_size_mode +2 ) and (font_size> file_font_size_mode or upper) and 5<len(head_check_text.strip())<80:                    
                        if exclusion_check(head_check_text,language) and not head_found: 
                            head_found=True
                            header_match_object=(font_size,font_family,bold,upper,bullet)
                            First_category=True
                            second_category=False
                            single_page_head_found=True
#                             print('FOUND Heading for exclusion ...',span.text)
                        
                    
                        elif ((font_size,font_family,bold,upper,bullet)==header_match_object  or font_size>header_match_object[0] ) and head_found and not exclusion_check(head_check_text,language):
                            head_found=False
                            single_page_head_found=True
#                             print('FOUND Closure for exclusion ...',span.text)
#                     if 'CLAIMS CONDITIONS' in span.text:
#                         print ('^^^^^^^',(font_size,font_family,bold,upper,bullet),header_match_object, font_size>header_match_object[0] , head_found , not exclusion_check(head_check_text,language))
#                         print(pointsRE_heading.findall(span.text))
                    words_title=[word.istitle() for word in head_check_text.split() if stopword_check(word,head_check_text,language) and not word.isdigit() ]
                    if span_position==0 and all(words_title) and head_check_text.strip() not in definitions and  (len(words_title) >1 or underlined_text )and 5<len(head_check_text.strip())<80 :
#                         print('MMMMMMMMARARARA',span.text)

                        if exclusion_check(head_check_text,language) and  not head_found:
                            head_found=True
                            second_category=True
                            First_category=False
                            single_page_head_found=True
                            header_match_object=(font_size,font_family,bold,upper,bullet)
#                             print("FOUND heading type2 ....",span.text)
                        elif head_found and not exclusion_check(head_check_text,language) and second_category and ((font_size,font_family,bold,upper,bullet)==header_match_object or font_size > header_match_object[0] ) :
                            head_found=False
                            second_category=False
                            single_page_head_found=True
#                             print('FOUND closure type 2',span.text)
                    if head_found:
                        excl_text=excl_text+' '+span.text     
                     
                    #CONDITIONS
#                     if 'Condition' in span.text:
# #                         print('text:',str(span.text),'head check',head_check_text,'length of head check',len(head_check_text))
# #                         print(span_position==0,(bold or font_size>= file_font_size_mode +2 ), (font_size> file_font_size_mode or upper), 5<len(head_check_text.strip())<80)
# #                         print(font_size,file_font_size_mode,cond_head_found,criteria_check(head_check_text,language))
                    if  span_position==0 and (bold or font_size>= file_font_size_mode +2 ) and (font_size> file_font_size_mode or upper) and 5<len(head_check_text.strip())<80:                    
                        if criteria_check(head_check_text,language) and not cond_head_found: 
                            cond_head_found=True
                            cond_header_match_object=(font_size,font_family,bold,upper,bullet)
                            cond_First_category=True
                            cond_second_category=False
                            cond_single_page_head_found=True
#                             print('FOUND Heading for condition...',span.text)
                    
                        elif ((font_size,font_family,bold,upper,bullet)==cond_header_match_object or font_size>cond_header_match_object[0] ) and not criteria_check(head_check_text,language)  and cond_head_found:
                            cond_head_found=False
                            cond_single_page_head_found=True
#                             print('FOUND Closure for condition ...',span.text)
                
                    
                    cond_words_title=[word.istitle() for word in head_check_text.split() if stopword_check(word,head_check_text,language) and not word.isdigit() ]
                    if span_position==0 and all(cond_words_title) and head_check_text.strip() not in definitions and  (len(words_title) >1 or underlined_text )and 5<len(head_check_text.strip())<80 :
#                         print('MMMMMMMMARARARA',span.text)

                        if criteria_check(head_check_text,language) and  not cond_head_found:
                            cond_head_found=True
                            cond_second_category=True
                            cond_First_category=False
                            cond_single_page_head_found=True
                            cond_header_match_object=(font_size,font_family,bold,upper,bullet)
#                             print("FOUND heading type2 ....",span.text)
                        elif cond_head_found and not criteria_check(head_check_text,language) and cond_second_category and ((font_size,font_family,bold,upper,bullet)==cond_header_match_object or font_size > cond_header_match_object[0] ) :
                            cond_head_found=False
                            cond_second_category=False
                            cond_single_page_head_found=True
#                             print('FOUND closure type 2',span.text)

                    if cond_head_found==True:
                        condition_text=condition_text+' '+span.text
                            
                    
                    #####EXTENSIONS
                    #CONDITIONS
#                     if 'Extension' in span.text:
#                         print(span.text)
#                         print('text:',str(span.text),'head check',head_check_text,'length of head check',len(head_check_text))
#                         print(span_position==0,(bold or font_size>= file_font_size_mode +2 ), (font_size> file_font_size_mode or upper), 5<len(head_check_text.strip())<80)
#                         print(font_size,file_font_size_mode,cond_head_found,criteria_check(head_check_text,language))
                    if  span_position==0 and (bold or font_size>= file_font_size_mode +2 ) and (font_size> file_font_size_mode or upper) and 5<len(head_check_text.strip())<80:                    
                        if ext_check(head_check_text,language) and not ext_head_found: 
                            ext_head_found=True
                            ext_header_match_object=(font_size,font_family,bold,upper,bullet)
                            ext_First_category=True
                            ext_second_category=False
                            ext_single_page_head_found=True
#                             print('FOUND Heading for ext...',span.text)
                    
                        elif ((font_size,font_family,bold,upper,bullet)==ext_header_match_object or font_size>ext_header_match_object[0] ) and not ext_check(head_check_text,language)  and ext_head_found:
                            ext_head_found=False
                            ext_single_page_head_found=True
#                             print('FOUND Closure for ext ...',span.text)
                
                    
                    ext_words_title=[word.istitle() for word in head_check_text.split() if stopword_check(word,head_check_text,language) and not word.isdigit() ]
                    if span_position==0 and all(ext_words_title) and head_check_text.strip() not in definitions and  (len(words_title) >1 or underlined_text )and 5<len(head_check_text.strip())<80 :
#                         print('MMMMMMMMARARARA',span.text)

                        if ext_check(head_check_text,language) and  not ext_head_found:
                            ext_head_found=True
                            ext_second_category=True
                            ext_First_category=False
                            ext_single_page_head_found=True
                            ext_header_match_object=(font_size,font_family,bold,upper,bullet)
#                             print("FOUND heading type2 ext....",span.text)
                        elif ext_head_found and not ext_check(head_check_text,language) and ext_second_category and ((font_size,font_family,bold,upper,bullet)==ext_header_match_object or font_size > ext_header_match_object[0] ) :
                            ext_head_found=False
                            ext_second_category=False
                            ext_single_page_head_found=True
#                             print('FOUND closure type 2 ext',span.text)
# 
                    if ext_head_found==True:
                        ext_text=ext_text+' '+span.text
                            
    #                     else:
###############################             
                        
                    if font_size_match:
                        if font_size<=file_font_size_mode-1:
                            text_lis.append(' ')
                    text=span.text.replace('\n',' ')
                    text_plain=text.replace('\n',' ')
                    for term in def_terms:
                        if term.lower().strip() in text.lower() and definitions[term][0].strip() not in text and term not in sub_definitions_in_page:
#                             def_page.append(definitions[term])
#                             print(pagenum)
                            src_str  = re.compile(re.escape(term), re.IGNORECASE)
                            text=src_str.sub('###{}@@{}%%%'.format(term,definitions[term][0]),text)
#                             print(text,'\n',term,'TERM!!!!@#$%^&*&^%#$%^&')
                            sub_definitions_in_page.append(term)
                            sub_page_def_list.append({'name':term,'text':definitions[term][0]})
#                             print(definitions_in_page)
                            break
#                     print(definitions_in_page)

                    
                    text_lis.append(text)
                    text_plain_lis.append(text_plain)
                    previous_span=span
                    

            if text_lis:
                if(int(pagenum) in [1,2]) and 'Wording' in file and pg_no >20:
#                     print('Continued on wording 2')
                    continue

                text_lis=text_lis[:-1]
                text_para=''.join(text_lis)
                text_plain_lis=text_plain_lis[:-1]
                text_plain_para=''.join(text_plain_lis)
                if(len(re.findall('\.',text_para))>(len(text_para)/3) or (len(re.findall('\d',text_para))>80 and int(pagenum)<4 and pg_no>20 and 'content' in text_para.lower())) :
                    print('PASS')
                    print(text_plain_para)
                    text_para,text_plain_para=' ',' '
                    text_lis,text_plain_lis=[],[]
                    single_page_head_found=False
                    head_found=False
                    cond_head_found=False
                    ext_head_found=False
                    continue
                else:
                    bold_lis=clean_pharses(bold_lis)
                    if pagenum in page_list and pagenum!='1':
                        pagenum=int(pagenum)+1
                    excl_flag,excl_count,excl_pos_lis=excl_extract(text_plain_lis,language)
                    cond_flag,cond_count,cond_pos_lis=cond_extract(text_plain_lis,language)
                    ext_flag,ext_count,ext_pos_lis=ext_extract(text_plain_lis,language)
                    if int(pagenum)==1:
                        if head_found or single_page_head_found or excl_count>0:
#                             print(pagenum,True,'!!!',excl_count)
                            excl_flag=True
                        else:
                            excl_flag=False
                    else:
                        if head_found or single_page_head_found:
#                             print(pagenum,True,'!!!')
                            excl_flag=True
#                             print(excl_flag,'EXCL')
                        
                    #####################
                    if cond_head_found or cond_single_page_head_found or cond_count>0:
                        cond_flag=True
#                         print('COND','TRUE')
                    else:
                        cond_flag=False
                    
                    if ext_head_found or ext_single_page_head_found or ext_count>0:
                        ext_flag=True
#                         print('EXT','TRUE',pagenum)
                    else:
#                         print('EXT','FALSE',pagenum)
                        ext_flag=False
#                         print('COND','FALSE')
#                     page_def_list.append({'name':term,'text':definitions[term][0]}
#                     def_in_page=[{'name':k,'text':v[0]} for k,v in definitions.items() if v[1]==pagenum]  
#                     print(def_in_page,'!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
                    for term,defs in definitions.items():
                        if defs[1]==pagenum:  ###defs[1] is the pagenumber
                            definition_text=definition_text+'          '+term+' '+defs[0]
                            def_in_page.append({'name':term,'text':defs[0]})        
#                     print(definition_text)
                    if definition_text:
                        def_flag=True

                    if def_flag:
                        def_search_flag=True
#                         print('DEF','TRUE')
                    else:
                        def_search_flag=False
#                         print('DEF','FALSE')
                    
                    if pg_no>2:
                        endorsements=False
                    else:
                        endorsements=True
#                     print(endorsements,'ENDORSEMENTS!!!')        
                    if text_para in local_indexed.keys():
                        text_para=text_para+' '
                    local_indexed[text_para]=(file,pagenum,bold_lis,text_plain_para,excl_flag,excl_count,excl_pos_lis,subdir,sub_page_def_list,country,language,definition_text,def_search_flag,endorsements,excl_text,condition_text,cond_flag,cond_count,cond_pos_lis,def_in_page,ext_text,ext_flag,ext_count,ext_pos_lis)
#                     print(sub_page_def_list,'DEF LIST!!!!!1')
                    sub_page_def_list=[]
                    text_para,text_plain_para='','' 
                    text_plain_lis,text_lis,bold_lis=[],[],[]
                    definition_text=''
                    single_page_head_found=False
                    cond_single_page_head_found=False
                    ext_single_page_head_found=False
                    condition_text=''
                    ext_text=''
                    excl_text=''
                    def_in_page=[]
                    def_flag=False
    return(local_indexed)

In [18]:
def doc_pdf_path(path):
    file_list=os.listdir(path)
    for file in file_list:
        raw_filename=file[:file.rfind(".")]
        pdf_filename=raw_filename+'.pdf'
#         print(raw_filename,pdf_filename,'FILE!!')
        if pdf_filename in file_list:
            print('continued!!!!!!!!!!!!!')
            continue
        if file.endswith('doc') or file.endswith('docx'):
            print('processing!!!')
            wordToPdf(os.path.join(path,file))

In [ ]:
# rootDir='V://COG//AU- Property//'
# for dirname,subdirlist,filelist in os.walk(rootDir):
#     print(dirname)
#     for subdir in subdirlist:
#         path=os.path.join(dirname,subdir)
# #         doc_pdf_path(path)

In [19]:
# import os
file_count=0
rootDir='Documents/SG-Property/'
country='singapore'
local_indexed={}
language='english'
if 'spanish' in rootDir:
    language='spanish'
for dirname,subdirlist,filelist in os.walk(rootDir):
    for subdir in subdirlist:
        print(subdir, 'SUBDIR!!!!!!!!!!!!!')
        path=os.path.join(dirname,subdir)    
        local_indexed_subdir=create_local_index(path,subdir,language,country)
        local_indexed.update(local_indexed_subdir)

Other Carrier Wording SUBDIR!!!!!!!!!!!!!
Broker Wording SUBDIR!!!!!!!!!!!!!
Chubb Standard Wording SUBDIR!!!!!!!!!!!!!
OUTSIDE LOCATION CLAUSE.pdf
COUNT 1
BIOLOGICAL OR CHEMICAL MATERIALS EXCLUSION.pdf
COUNT 2
DEFINITIONS.pdf
COUNT 3
FOREIGN ENTITY LOSS CLAUSE.pdf
COUNT 4
COLLAPSE OF AERIAL CLAUSE.pdf
COUNT 5
HEATING AND POWER CLAUSE.pdf
COUNT 6
DIFFERENCE IN CONDITIONS DIFFERENCE IN LIMITS.pdf
COUNT 7
BREACH OF WARRANTIES CLAUSE.pdf
COUNT 8
SMOKE DAMAGE.pdf
COUNT 9
DIVISIBLE CONTROL CLAUSE.pdf
COUNT 10
ELECTRONIC DATE RECOGNITION EXCLUSION (EDRC) (A).pdf
COUNT 11
AWNINGS, BLINDS, SIGNS AND OUTDOOR FIXTURES OR FITTINGS OF ANY DESCRIPTION.pdf
COUNT 12
AUTOMATIC COVER FOR NEW LOCATIONS.pdf
COUNT 13
CLAIMS CO-OPERATION CLAUSE.pdf
COUNT 14
INTERLOCKING CLAUSE (CHARTIS_SPECIMEN).pdf
COUNT 15
INLAND TRANSIT (ALL RISKS) CLAUSE.pdf
COUNT 16
APPROVED ADJUSTERS - CUNNINGHAM AND INSIGHT.pdf
COUNT 17
Excluded Clauses.pdf
COUNT 18
ARMED ROBBERY AND OR HOLD UP CLAUSE.pdf
COUNT 19
DEBRIS REMOVAL.pdf

Electronic Equipment and or Device Endorsement.pdf
COUNT 181
COST OF TEMPORARY PROTECTION CLAUSE.pdf
COUNT 182
CLAIMS CO-OPERATION CLAUSE (Bowring Marsh).pdf
COUNT 183
ALTERATIONS AND ADDITIONS CLAUSE.pdf
COUNT 184
BREACH OF CONDITIONS CLAUSE.pdf
COUNT 185
AGREED VALUE CLAUSE.pdf
COUNT 186
ELECTRICAL INSTALLATION CLAUSE 4A.pdf
COUNT 187
CO INSURANCE CLAUSE.pdf
COUNT 188
REMOVAL OF DEBRIS CLAUSE.pdf
COUNT 189
FULL THEFT (FIRST LOSS LIMIT).pdf
COUNT 190
Con Loss Wording.pdf
COUNT 191
LOSS OF ATTRACTION EXTENSION.pdf
COUNT 192
WARRANTY 1.pdf
COUNT 193
NON CANCELLATION CLAUSE.pdf
COUNT 194
BURGLARY ENDORSEMENT (FIRST LOSS LIMIT).pdf
COUNT 195
FIRE EXTINGUISHING COSTS.pdf
COUNT 196
EMPLOYEESÔÇÖ PERSONAL EFFECTS.pdf
COUNT 197
APPROVED ADJUSTERS CLAUSE.pdf
COUNT 198
AUTOMATIC INCLUSION OF NEW LOCATIONS.pdf
COUNT 199
VANDALISM AND MALICIOUS MISCHIEF ENDORSEMENT.pdf
COUNT 200
CONTROL OF PROPERTY.pdf
COUNT 201
SPECIAL CONDITIONS FOR DECLARATION POLICIES.pdf
COUNT 202
CO-INSURANCE CLAUSE - LEAD C

In [20]:
len(local_indexed)

347

In [21]:
from azure.cosmos import exceptions, CosmosClient, PartitionKey

cosmo_endpoint="https://nf-poc-cdb-sql.documents.azure.com"
cosmo_key="iEcEfrxYe0Fm9QtoxDrOpLvGsfzjowwybULlWT9Uz4XxV4RmOIAnRuLdgRFUu1LPU5Vwk3UGivRrPrxnk7083w=="
client = CosmosClient(cosmo_endpoint, cosmo_key)
database=client.get_database_client('policy-analysis')
container=database.get_container_client('property-wordings')

In [22]:
for key,value in local_indexed.items():
    doc={}
    doc['text']=key
    doc['page']=int(value[1])
    if value[4]==True or value[0].find('Exclusion')!=-1:
        doc['IsExclusion']=True
    else:
        doc['IsExclusion']=False
    doc['doc_name']=value[0].replace('.pdf','')
    doc['id']=doc['doc_name']+' '+str(doc['page'])
    doc['bold_phrases']=value[2]
    doc['plain_text']=value[3]
    doc['excl_count']=value[5]
    doc['excl_pos']=value[6]
    doc['folder']=value[7]
    doc['definitions']=[{'name':key,'text':value} for key,value in re.findall('###(.*?)@@(.*?)%%%',doc['text'])]
    doc['country']=value[9]
    doc['language']=value[10]
    doc['definition_text']=value[11]
#     print(doc['definition_text'],'\n\n')
    if doc['definition_text']:
        doc['definition_flag']=True
    else:
        doc['definition_flag']=False
    if value[14]==True or 'endorsement' in value[0].lower(): 
        doc['endorsements']=True
    else:
        doc['endorsements']=value[13]
    doc['excl_text']=value[14]
    doc['cond_text']=value[15]
    doc['cond_flag']=value[16]
    doc['cond_count']=value[17]
    doc['cond_pos']=value[18]
    doc['definitions_in_page']=value[19]
    doc['ext_text']=value[20]
    doc['ext_flag']=value[21]
    doc['ext_count']=value[22]
    doc['ext_pos']=value[23]
    
    try:       
        container.create_item(body=doc)
    except Exception as err:
        print(err)
        pass

(Conflict) Entity with the specified id already exists in the system., 
RequestStartTime: 2020-12-16T16:12:09.3759829Z, RequestEndTime: 2020-12-16T16:12:09.3759829Z,  Number of regions attempted:1
ResponseTime: 2020-12-16T16:12:09.3759829Z, StoreResult: StorePhysicalAddress: rntbd://cdb-ms-prod-eastus1-fd58.documents.azure.com:14073/apps/f731575d-fb29-4640-9ffc-a6de234ddc91/services/78ee78c6-b247-456d-ab37-8e45873f3782/partitions/0e690849-4cb2-4a3c-b969-080feeb81a53/replicas/132525678004960632p/, LSN: 575, GlobalCommittedLsn: 575, PartitionKeyRangeId: 0, IsValid: True, StatusCode: 409, SubStatusCode: 0, RequestCharge: 1.67, ItemLSN: -1, SessionToken: -1#575, UsingLocalLSN: False, TransportException: null, ResourceType: Document, OperationType: Create
, Microsoft.Azure.Documents.Common/2.11.0
(Conflict) Entity with the specified id already exists in the system., 
RequestStartTime: 2020-12-16T16:12:10.6259823Z, RequestEndTime: 2020-12-16T16:12:10.6259823Z,  Number of regions attempted:1


In [23]:
print('added to cosmos')

added to cosmos
